In [41]:
import pandas as pd

In [42]:
df_1 = pd.read_csv('data/training.csv', index_col='Date')
df_2 = pd.read_csv('data/testing.csv')

In [5]:
df_1.shape

(7806, 49)

In [6]:
df_2.shape

(876, 49)

In [35]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

TRAINING_DATASET_PATH = "data/training.csv"
MODEL_PATH = "model.pkl"
OUTPERFORMANCE = 10

def status_calc(stock, sp500):
    """A simple function to classify whether a stock outperformed the S&P500
    :param stock: stock price
    :param sp500: S&P500 price
    :return: true/false
    """
    return stock - sp500 >= OUTPERFORMANCE

def build_data_set():
    """
    Reads the keystats.csv file and prepares it for scikit-learn
    :return: X_train and y_train numpy arrays
    """
    training_data = pd.read_csv(TRAINING_DATASET_PATH, index_col="Date")

    training_data.dropna(axis=0, how="any", inplace=True)
    features = training_data.columns[7:]
    print(features)
    X_train = training_data[features].values
    #print(X_train.columns)
    print('hello')
    # Generate the labels: '1' if a stock beats the S&P500 by more than 10%, else '0'.
    y_train = list(
        status_calc(
            training_data["stock_p_change"],
            training_data["SP500_p_change"]
        )
    )
    return X_train, y_train

def train_model():
    X_train, y_train = build_data_set()
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    
    return model

In [39]:
model = train_model()

Index(['Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E',
       'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Profit Margin', 'Operating Margin',
       'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share',
       'Qtrly Revenue Growth', 'Gross Profit', 'EBITDA',
       'Net Income Avl to Common', 'Diluted EPS', 'Qtrly Earnings Growth',
       'Total Cash', 'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity',
       'Current Ratio', 'Book Value Per Share', 'Operating Cash Flow',
       'Levered Free Cash Flow', 'Beta', '50-Day Moving Average',
       '200-Day Moving Average', 'Avg Vol (3 month)', 'Shares Outstanding',
       'Float', '% Held by Insiders', '% Held by Institutions',
       'Shares Short (as of', 'Short Ratio', 'Short % of Float',
       'Shares Short (prior month)'],
      dtype='object')
hello


In [40]:
df = pd.read_csv('data/testing.csv', index_col="Date")
features = df.columns[6:]
X_test = df[features].values
predictions = model.predict(X_test)
df['Date'] = df.index

predictions_series = pd.Series(predictions, name='Predictions', index=df.index)

# Concatenate predictions with the original DataFrame
final = pd.concat([df[['Date', 'Ticker', 'stock_p_change', 'SP500_p_change']], predictions_series], axis=1)

ValueError: X has 42 features, but RandomForestClassifier is expecting 41 features as input.

In [21]:
final

,Date,Ticker,stock_p_change,SP500_p_change,Predictions
Date,,,,,
2013-04-26 17:42:40,2013-04-26 17:42:40,ctas,30.78,20.10,False
2013-09-06 01:19:11,2013-09-06 01:19:11,vz,12.60,23.50,False
2013-02-06 09:42:13,2013-02-06 09:42:13,vz,8.85,19.78,False
2013-04-25 05:46:09,2013-04-25 05:46:09,vz,-9.86,19.89,False
2013-05-21 06:58:47,2013-05-21 06:58:47,gme,3.30,15.42,False
...,...,...,...,...,...
2013-05-08 10:24:27,2013-05-08 10:24:27,nyx,12.41,17.22,False
2013-04-22 22:47:52,2013-04-22 22:47:52,nyx,20.52,22.74,False
2013-05-13 18:46:02,2013-05-13 18:46:02,xl,3.50,18.50,False


In [25]:
from fastapi.responses import JSONResponse
import json
# Convert the final DataFrame to JSON
final_json = final.to_json(orient="records")

# Return the JSON response
#content=json.loads(final_json)
final_json

'[{"Date":"2013-04-26 17:42:40","Ticker":"ctas","stock_p_change":30.78,"SP500_p_change":20.1,"Predictions":false},{"Date":"2013-09-06 01:19:11","Ticker":"vz","stock_p_change":12.6,"SP500_p_change":23.5,"Predictions":false},{"Date":"2013-02-06 09:42:13","Ticker":"vz","stock_p_change":8.85,"SP500_p_change":19.78,"Predictions":false},{"Date":"2013-04-25 05:46:09","Ticker":"vz","stock_p_change":-9.86,"SP500_p_change":19.89,"Predictions":false},{"Date":"2013-05-21 06:58:47","Ticker":"gme","stock_p_change":3.3,"SP500_p_change":15.42,"Predictions":false},{"Date":"2013-04-22 01:30:37","Ticker":"gme","stock_p_change":33.68,"SP500_p_change":22.74,"Predictions":false},{"Date":"2013-02-06 13:03:37","Ticker":"amzn","stock_p_change":35.23,"SP500_p_change":19.78,"Predictions":false},{"Date":"2013-09-05 21:27:13","Ticker":"amzn","stock_p_change":17.78,"SP500_p_change":23.56,"Predictions":true},{"Date":"2013-05-08 15:04:22","Ticker":"amzn","stock_p_change":11.46,"SP500_p_change":17.22,"Predictions":fal